## Варіант 5
### Завдання  1. Обчислити заданий інтеграл Рімана із точністю 0,001:
- методом трапецій із визначенням кількості інтервалів розбиття через оцінку похибки;
- методом Сімпсона із використання принципу Рунге;
- за допомогою формул Гауса.

За отриманими результатами обчислень зробити висновки.

| № варіанту |        $f (x)$        | a | b |
|:----------:|:---------------------:|:---:|:---:|
|      5     | $$\frac{sin x}{x^2+1}$$ |  0  |  1  |

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import display, Markdown, Math
from matplotlib.pyplot import figure
from numpy.polynomial import polynomial
from scipy.misc import derivative

def init_fig():
    global figure
    figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')

In [3]:
f = lambda x: np.sin(x)/(x**2 + 1)
a = 0
b = 1
error = 0.001

In [89]:
class Integral():
    """Inegral to numericaly solve different methods"""
    def __init__(self, func, a, b, error = 0.001):
        """
        func : function to integrate
        a : left bound
        b : right bound
        """
        self.func = func
        self.a = a
        self.b = b
        self.error = error
    
    def max_der(self, order):
        M = np.abs(
            derivative(
                self.func,
                np.linspace(self.a, self.b, 100),
                n=order,
                dx=0.0005,
                order=order+1+order%2
            )
        ).max()
        return M
    
    def runge_error(self, step_integral, halfstep_integral, accuracy_order):
        return np.abs(step_integral - halfstep_integral)/(2**accuracy_order-1)
        

class TrapezoidalIntegral(Integral):
    def distance_between_points(self):
        h = np.sqrt(
            12*self.error/(self.max_der(2)*np.abs(self.b-self.a))
        )
        return h
    
    def interval_number(self):
        n = int(
            np.ceil(
                np.abs(self.b - self.a)/self.distance_between_points()
            )
        )
        return n
    
    def exact_error(self, h):
        """Trapezoidal integral"""
        return np.abs(self.max_der(2)*np.abs(self.b - self.a)*h**2/12)
    
    def solve(self):
        n = self.interval_number()
        h = (self.b - self.a)/n
        f_x_i = self.func(np.linspace(self.a, self.b, n+1))
        # for trapezional formula
        f_x_i[0] /= 2
        f_x_i[-1] /= 2
        answer = h*np.sum(f_x_i)
        return answer, self.exact_error(h)


class SimpsonIntegral(Integral):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x_i = np.linspace(self.a, self.b, 3)
        self.y_i = self.func(self.x_i)
        self.subdivision_number = 0
    
    def simpson_runge_error(self, step_integral, halfstep_integral):
        return self.runge_error(step_integral, halfstep_integral, 4)

    def subdivide(self):
        """Subdivide self.x_i and self.y_i"""
        # make one more x_i betwen every nearest x_i
        self.x_i = np.linspace(self.a, self.b, (self.x_i.size-1)*2+1)
        old_y_i = self.y_i
        # concatenate i. e.
        # from [f1, f2, f3, ..., f99, f100]
        # to   [f1, f2, f2, ..., f97, f98, f98, f99, f99, f99, f100]
        self.y_i = np.concatenate([
            old_y_i[:1],
            np.repeat(old_y_i[1:-1], 2),
            old_y_i[-2:],
        ])
        self.y_i[1:-1:2] = self.func(self.x_i[1:-1:2])
    
    def simpsons_rule(self):
        n = self.x_i.size - 1
        h = (self.x_i[-1] - self.x_i[0])/n
        # instead adding two sums from y_i we modify temp value according to the rule 
        intermediate_y_i = np.copy(self.y_i) 
        intermediate_y_i[1:-1:2] *= 4
        intermediate_y_i[2:-1:2] *= 2
        return h/3 * np.sum(intermediate_y_i)
    
    def iter_runge_solve(self):
        """
        Subdivide to that point when Runge error will satisfy the condition error
        """
        prev_answer = self.simpsons_rule()
        self.subdivide()
        self.subdivision_number += 1
        curr_answer = self.simpsons_rule()
        while self.simpson_runge_error(prev_answer, curr_answer) > self.error:
            self.subdivide()
            self.subdivision_number += 1
            prev_answer, curr_answer = curr_answer, self.simpsons_rule()
        return curr_answer, self.simpson_runge_error(prev_answer, curr_answer)

trap_int = TrapezoidalIntegral(f, a, b, error)
trapezoidal_answer, trapezoidal_error = trap_int.solve()

simp_int = SimpsonIntegral(f, a, b, error*0.1)
simpsons_answer, simpsons_error = simp_int.iter_runge_solve()